# Assay Comparison
Review assay summary statistics between badapple (original) and badapple_classic DB. 

## Setup

In [17]:
# define DB variables
ORIGINAL_DB_NAME = "badapple"
ORIGINAL_DB_HOST = "localhost"
ORIGINAL_DB_USER = "<your_usr>"
ORIGINAL_DB_PASSWORD = "<your_pw>"

COMPARISON_DB_NAME = "badapple_classic"
COMPARISON_DB_HOST = ORIGINAL_DB_HOST
COMPARISON_DB_USER = ORIGINAL_DB_USER
COMPARISON_DB_PASSWORD = ORIGINAL_DB_PASSWORD

In [18]:
# imports
import psycopg2
import psycopg2.extras
from psycopg2 import sql
import pandas as pd

## Connect to DBs

In [19]:
def db_connect(db_name: str, host: str, user: str, password: str):
    try:
        db_connection = psycopg2.connect(
            dbname=db_name,
            host=host,
            user=user,
            password=password,
            cursor_factory=psycopg2.extras.DictCursor,
        )
        db_connection.set_session(readonly=True)
        return db_connection
    except Exception as e:
        print(e)
        print(f"Error connecting to DB: {db_name}")
        return None

In [20]:
original_db_connection = db_connect(ORIGINAL_DB_NAME, ORIGINAL_DB_HOST, ORIGINAL_DB_USER, ORIGINAL_DB_PASSWORD)
comparison_db_connection = db_connect(COMPARISON_DB_NAME, COMPARISON_DB_HOST, COMPARISON_DB_USER, COMPARISON_DB_PASSWORD)

In [21]:
original_db_cur = original_db_connection.cursor()
comparison_db_cur = comparison_db_connection.cursor()

## View summary statistics of each DB

In [22]:
def get_summary_stats(db_cursor, db_conn):
    query = sql.SQL("SELECT median_ncpd_tested,median_nsub_tested,median_nass_tested,median_nsam_tested,nass_total FROM metadata")
    result = []
    try:
        db_cursor.execute(query)
        result = db_cursor.fetchall()
        return result[0] # only 1 row
    except Exception:
        db_conn.rollback()
    return result

In [23]:
original_stats = get_summary_stats(original_db_cur, original_db_connection)
comparison_stats = get_summary_stats(comparison_db_cur, comparison_db_connection)

In [24]:
columns = ["median_ncpd_tested","median_nsub_tested","median_nass_tested","median_nsam_tested","nass_total"]
original_summary_df = pd.DataFrame([original_stats], columns=columns)
comparison_summary_df = pd.DataFrame([comparison_stats], columns=columns)

In [25]:
original_summary_df

,median_ncpd_tested,median_nsub_tested,median_nass_tested,median_nsam_tested,nass_total
0,2,2,633,774,823


In [26]:
comparison_summary_df

,median_ncpd_tested,median_nsub_tested,median_nass_tested,median_nsam_tested,nass_total
0,2,2,632,773,823


## Close DB connections

In [27]:
# Close the cursors
original_db_cur.close()
comparison_db_cur.close()

# Close the connections
original_db_connection.close()
comparison_db_connection.close()